In [7]:
import pandas as pd
import pickle
from utils import load_data
from scipy.sparse import coo_matrix
import seaborn as sns
import numpy as np

In [2]:
browsing, search, sku, info = load_data('../dataset/new/')

load browsing done...
load search done...
load sku done...
load info done...


In [3]:
search.columns

Index(['session_id_hash', 'query_vector', 'clicked_skus_hash',
       'product_skus_hash', 'server_timestamp_epoch_ms'],
      dtype='object')

In [4]:
info.idx2act

{0: 'detail', 1: 'pageview', 2: 'add', 3: 'purchase', 4: 'remove'}

In [5]:
brwosing = browsing[browsing['product_action'] != 4]

In [8]:
url2item = browsing.dropna().drop_duplicates(subset=['hashed_url', 'product_sku_hash']).groupby('hashed_url')['product_sku_hash'].agg(list)
url2item = url2item.reset_index()
url2item['product_sku_hash'] = url2item['product_sku_hash'].apply(lambda x: x[0])
url2item = np.vstack(url2item.values).T
url2item = url2item.astype(int).tolist()
url2item = dict(zip(*url2item))
browsing['product_sku_hash'] = browsing['hashed_url'].map(url2item.get)

# add click data

In [6]:
# search_click = search.explode("clicked_skus_hash")
# search_click = search_click.dropna(subset=['clicked_skus_hash'])
# search['item_count'] = search.groupby('session_id_hash')['clicked_skus_hash'].transform('count')
# search = search.sort_values(by=['session_id_hash', 'server_timestamp_epoch_ms'])

In [7]:
# search_view = search.explode("product_skus_hash")
# search_view = search_view.dropna(subset=['product_skus_hash'])

In [8]:
# search_click = search_click[['session_id_hash', 'clicked_skus_hash', 'server_timestamp_epoch_ms']]
# search_click.columns = ['sess_id', 'sku_id', 'timestamp']

In [9]:
# search_view = search_view[['session_id_hash', 'product_skus_hash', 'server_timestamp_epoch_ms']]
# search_view.columns = ['sess_id', 'sku_id', 'timestamp']

In [10]:
# search = pd.concat([search_click, search_view])
# search.drop_duplicates(subset=['sess_id', 'sku_id'], keep='last', inplace=True)

In [11]:
# search_train =  search.loc['train', :].copy()
# search_test = search.loc['test', :].copy()

In [9]:
browsing.dropna(subset=['product_sku_hash'], inplace=True)
browsing.drop_duplicates(subset=['session_id_hash', 'product_sku_hash'], keep='last', inplace=True)
# browsing = browsing.sort_values(by=['session_id_hash', 'server_timestamp_epoch_ms'])
browsing['product_sku_hash'] = browsing['product_sku_hash'].astype('int')

In [10]:
browsing = browsing[['session_id_hash', 'product_sku_hash', 'server_timestamp_epoch_ms']]
browsing.columns = ['sess_id', 'sku_id', 'timestamp']

In [11]:
browsing_train = browsing.loc['train', :].copy()
browsing_test = browsing.loc['test', :].copy()

In [12]:
# train = pd.concat([browsing_train, search_train])
# test = pd.concat([browsing_test, search_test])
train = browsing_train.copy()
test = browsing_test.copy()

In [13]:
test.drop_duplicates(subset=['sess_id', 'sku_id'], keep='last', inplace=True)
train.drop_duplicates(subset=['sess_id', 'sku_id'], keep='last', inplace=True)

In [14]:
train['item_count'] = train.groupby('sess_id')['sku_id'].transform('count')
train = train[train['item_count'] >=2]
train.drop(columns=['item_count'], inplace=True)

In [15]:
# test['item_count'] = test.groupby('sess_id')['sku_id'].transform('count')
# test = test[test['item_count'] >=2]
# test.drop(columns=['item_count'], inplace=True)

In [16]:
test_pad = test.groupby('sess_id').tail(1).copy()
test_pad['timestamp'] += 1
test_dev = pd.concat([test, test_pad])

In [17]:
test_dev = test_dev.sort_values(by=['sess_id', 'timestamp'])
train = train.sort_values(by=['sess_id', 'timestamp'])

In [19]:
train_set = []
valid_set = []
test_set = []

In [20]:
seq_len = 5
for sess_id, item_ids in train.groupby('sess_id')['sku_id']:
    item_ids = item_ids.tolist()
    for idx in range(1, len(item_ids)-1):
        if idx < seq_len:
            seq = item_ids[:idx] + [-1] * (seq_len - idx)
        else:
            seq = item_ids[idx - seq_len:idx]
        target = item_ids[idx]
        train_set.append((sess_id, seq, target))
        
    idx = len(item_ids)-1
    if idx < seq_len:
        seq = item_ids[:idx] + [-1] * (seq_len - idx)
    else:
        seq = item_ids[idx - seq_len:idx]
    target = item_ids[idx]
    valid_set.append((sess_id, seq, target))  

In [21]:
for sess_id, item_ids in test_dev.groupby('sess_id')['sku_id']:
    item_ids = item_ids.tolist()
    for idx in range(1, len(item_ids)-1):
        if idx < seq_len:
            seq = item_ids[:idx] + [-1] * (seq_len - idx)
        else:
            seq = item_ids[idx - seq_len:idx]
        target = item_ids[idx]
        train_set.append((sess_id, seq, target))
        
    idx = len(item_ids)-1
    if idx < seq_len:
        seq = item_ids[:idx] + [-1] * (seq_len - idx)
    else:
        seq = item_ids[idx - seq_len:idx]
    target = item_ids[idx]
    test_set.append((sess_id, seq, target))  

In [22]:
len(train_set), len(valid_set), len(test_set)

(3395175, 1512420, 96108)

In [23]:
for name, data in zip(['train', 'valid', 'test'], [train_set, valid_set, test_set]):
    pathname = f'../dataset/prepared/{name}.csv'
    with open(pathname, 'w') as f:
        for user_id, seq, target in data:
            f.write(','.join([str(user_id), '|'.join(map(str, seq)), str(target)]))
            f.write('\n')